<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_gpt_9300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [ ]:
# !pip install konlpy

In [ ]:
# !pip install transformers

In [ ]:
# !pip install optuna

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

In [3]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
# DATA_PATH = "C:\python-code\project_final\data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

In [6]:
# gpt_neg = pd.read_csv(f"{DATA_PATH}gpt_neg_reviews_4400.csv")
gpt_neg = pd.read_csv(f"{DATA_PATH}gpt_neg_9300.csv")
gpt_neg

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9318,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [8]:
print(gpt_neg[gpt_neg['gpt_reply'].str.contains("사장님의 답변이 없습니다 ,")].index)

Int64Index([], dtype='int64')


In [12]:
# 결측치 확인
gpt_neg.isnull().sum()

original_review    0
original_reply     0
rating             0
gpt_review         0
gpt_reply          0
sentiment          0
score              0
dtype: int64

In [13]:
# 긍정 데이터 확인
gpt_neg[gpt_neg['sentiment'] == '긍정']

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score


In [14]:
# 부정 -> 0 으로 치환
gpt_neg['sentiment'] = 0
print(gpt_neg['sentiment'].sum()) # 합은 0
gpt_neg[gpt_neg['sentiment'] == '부정']

0


,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score


In [16]:
# 사용할 컬럼만 가져오기
cols = gpt_neg.columns[3:6] # 사용할 컬럼(gpt_review, gpt_reply, sentiment)만 가져오기
train_gpt = gpt_neg[cols]
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


## 데이터 정제

- 특이 패턴 텍스트(' , ') 제거

In [17]:
train_gpt['gpt_reply'][3640]

'풀리지 않고 바삭하지 않은 닭강정에 실망하셨다니 저희 부족한 부분을 반성하게 됐습니다. 앞으로는 보다 좋은 맛의 닭강정을 제공할 수 있도록 노력하겠습니다. '

In [ ]:
text = train_gpt['gpt_reply'][3640]

pattern = r'^\s*,\s*'
result = re.sub(pattern, '', text)

print(result)

고기가 신선하지 않아서 냄새가 나셨다니 정말 죄송합니다. 저희는 항상 청결과 신선한 재료로 요리를 준비하는 데 신경을 써왔는데, 미리 조치하도록 하겠습니다. , 


In [ ]:
reply_list = []
for reply in train_gpt['gpt_reply']:
    if isinstance(reply, str):
        pat = r'^\s*,\s*'
        result = re.sub(pattern, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

train_gpt['gpt_reply'] = reply_list
train_gpt.head()

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


In [ ]:
train_gpt['gpt_reply'][3640]

'고기가 신선하지 않아서 냄새가 나셨다니 정말 죄송합니다. 저희는 항상 청결과 신선한 재료로 요리를 준비하는 데 신경을 써왔는데, 미리 조치하도록 하겠습니다. , '

- 특이 답변 제거

In [18]:
train_gpt[train_gpt['gpt_reply'].str.contains("사장님의 답변이 없습니다 ,")]

,gpt_review,gpt_reply,sentiment


In [ ]:
train_gpt = train_gpt[~train_gpt['gpt_reply'].str.contains("사장님의 답변이 없습니다 ,")]
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0


In [ ]:
train_gpt.iloc[3700]

gpt_review     음식이 촉촉하지 않고 맛이 없어서 별로였어요. 너무 실망스러웠어요. 다음에는 절대...
gpt_reply      맛이 부족했고 실망스러웠던 점 정말 죄송합니다. 다음번에는 더 맛있는 음식을 제공...
sentiment                                                     0
Name: 3710, dtype: object

In [19]:
len(train_gpt['gpt_review']) , len(train_gpt['gpt_reply'])

(9323, 9323)

- 공백으로 시작하는 텍스트 제거

In [ ]:
reply_list = []
for reply in train_gpt['gpt_reply']:
    if isinstance(reply, str):
        pat = r'^\s*'
        result = re.sub(pat, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

train_gpt['gpt_reply'] = reply_list
train_gpt.head()

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


In [ ]:
review_list = []
for review in train_gpt['gpt_review']:
    if isinstance(review, str):
        pat = r'^\s*'
        result = re.sub(pat, '', review)
        review_list.append(result)
    else:
        review_list.append("")

train_gpt['gpt_review'] = review_list
train_gpt.head()

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


- 영어 답변 제거

In [20]:
train_gpt['gpt_reply'][2500]

'맛이 없어서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만나뵐 수 있도록 노력하겠습니다. '

In [ ]:
train_gpt[train_gpt['gpt_reply'].str.match('^[a-zA-Z]')]

,gpt_review,gpt_reply,sentiment
2494,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","Dear customer, we apologize for the unpleasant...",0
2495,양이 너무 적고 서비스도 별로였어요.,Thank you for your feedback. We apologize for ...,0
2496,"Delicious, but the delivery was extremely slow.",We re sorry to hear that the delivery was slow...,0
2497,피자가 맛없고 느끼하고 배달이 너무 느렸어요,We apologize for the tasteless and heavy pizza...,0
2498,항상 실망시키네요. 다음에는 안시킬 거에요,We re sorry to have consistently disappointed ...,0
2499,맛이 좀 없어서 먹기가 그렇게 좋지 않았어요,We apologize for the lack of flavor in the foo...,0
2500,The pizza was cold and tasteless. I won t be o...,We re sorry to hear that the pizza arrived col...,0
2501,구성도 별로였고 배달도 엄청 늦었어요. 다신 시키지 않을 거에요,We apologize for the poor composition of the f...,0
2502,배달이 너무 느렸고 맛도 그냥 그랬어요,We apologize for the slow delivery and average...,0
2503,맛있지 않고 서비스도 별로였어요.,We re sorry to hear that our food and service ...,0


In [ ]:
train_gpt = train_gpt[~train_gpt['gpt_reply'].str.match('^[a-zA-Z]')]
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비스...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 음...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있도...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하실...,0


# 저장

In [21]:
train_gpt.to_csv(f'{DATA_PATH}train_gpt_9300.csv', index=False, encoding = "utf-8-sig")
train_gpt.to_csv('train_gpt_9300.csv', index=False, encoding = "utf-8-sig")

In [22]:
# 엑셀로도 저장해서 확인해보기
with pd.ExcelWriter("train_gpt_9300.xlsx", engine='openpyxl') as writer:
    train_gpt.to_excel(writer, sheet_name='시트1', index=False)